In [ ]:
# Data Acquisition 

#Let's start with imports:

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [ ]:
#Let's define a method that claims html form a url

In [ ]:
def claim_html(url):
    user_agent = {'User-agent': '*'}
    res = requests.get(url, headers=user_agent)
    return BeautifulSoup(res.content,"html.parser")

In [ ]:
#bring our main domain

In [ ]:
main_page = claim_html("https://www.metacritic.com/browse/games/score/metascore/all/all/filtered")

In [ ]:
#claim our next page on main domain

In [ ]:
next_page = main_page.find("span",attrs={"class":"flipper next"}).a['href']

In [ ]:
#Variables declaration :
#each list for each 

In [ ]:
counter = 0
game_title = []
release_year =[]
num_of_critics=[]
critics_grade=[]
game_summary=[]
user_grade=[]
platform =[]
num_of_users = []
game_num_of_players = []
game_developer = []
df=pd.DataFrame()

In [1]:
#the main function:
#here the General explanation for main function and data extraction:

#This script gather the domains of games with release year of 2010 and above reaches their HTML page and 
#fetch the requested data using try and except in order to scrape MetaCritic and fill the lists.
#we added a counter to see our progress through each next main pages and we used the function sleep in 
#intervals of each request.


#This function also combines all DataFrames together into a one big DataFrame


In [ ]:
while next_page != None:
    try:
        game_info_list=main_page.findAll("td",attrs={"class":"clamp-summary-wrap"})
    except:
        time.sleep(6)
        game_info_list=main_page.findAll("td",attrs={"class":"clamp-summary-wrap"})
    for td in game_info_list:
        sp = td.find("div",attrs={"class":"clamp-details"}).select("span")
        release_date_string= str(sp[2].text)
        release_year_string=release_date_string[len(release_date_string)-4]+release_date_string[len(release_date_string)-3]+release_date_string[len(release_date_string)-2]+release_date_string[len(release_date_string)-1]
        if int(release_year_string)>=2010:
            if sp[1].text != "iOS":
                try:
                    game_page=claim_html("https://www.metacritic.com/"+td.find("a",attrs={"class":"title"})['href'])
                except:
                    time.sleep(6)
                    game_page=claim_html("https://www.metacritic.com/"+td.find("a",attrs={"class":"title"})['href'])
                try:
                    game_title.append(game_page.find("div",attrs={"class":"product_title"}).a.text.replace("\n",""))
                except:
                     game_title.append(None)
                release_year.append(release_year_string)
                platform.append((sp[1].text.replace("\n","")))
                try:
                    num_of_critics.append(game_page.find("span",attrs={"class":"count"}).a.span.text.replace("\n",""))
                except:
                    num_of_critics.append(None)
                try:
                    critics_grade.append(game_page.find("div",attrs={"class":"metascore_w xlarge game positive"}).text)
                except:
                    try:
                        critics_grade.append(game_page.find("div",attrs={"class":"metascore_w xlarge game mixed"}).text)
                    except:
                        try:
                            critics_grade.append(game_page.find("div",attrs={"href = ":"metascore_w xlarge game negative"}).text)
                        except:
                            critics_grade.append(None)
                try:
                    game_summary.append(game_page.find("span",attrs={"class":"blurb blurb_expanded"}).text)
                except:
                    game_summary.append(None)
                try:
                    game_developer.append(game_page.find("li", attrs={"class": "summary_detail developer"}).find("span", attrs={"class": "data"}).text.replace("\n",""))
                except:
                    game_developer.append(None)

                try:
                    num_of_users.append(game_page.find("div", attrs={"class":"userscore_wrap feature_userscore"}).find("div", attrs={"class": "summary"}).find("span",attrs={"class": "count"}).a.text)
                except:
                    num_of_users.append(None)

                try:
                    game_num_of_players.append(game_page.find("li",attrs={"class":"summary_detail product_players"}).find("span",attrs={"class": "data"}).text.replace("\n",""))
                except:
                    game_num_of_players.append(None)
                try:
                    user_grade.append(game_page.find("div",attrs={"class":"metascore_w user large game positive"}).text)
                except:
                    try:
                        user_grade.append(game_page.find("div",attrs={"class":"metascore_w user large game mixed"}).text)
                    except:
                        try:
                            user_grade.append(game_page.find("div",attrs={"class":"metascore_w user large game negative"}).text)
                        except:
                            user_grade.append(None)

                time.sleep(5)
    counter+=1
    try:
        main_page = claim_html("https://www.metacritic.com/"+next_page)
    except:
        time.sleep(6)
        main_page = claim_html("https://www.metacritic.com/"+next_page)
    if main_page.find("span",attrs={"class":"flipper next"}).a == None:
        next_page = None
    else:
        next_page=main_page.find("span",attrs={"class":"flipper next"}).a['href']
        if "browse" not in next_page:
            next_page = "browse"+next_page
    df_1=pd.DataFrame({"game_title":game_title,"release_year":release_year,"num_of_critics":num_of_critics,"critics_grade":critics_grade,"game_summary":game_summary,
                 "user_grade":user_grade,"platform":platform,"num_of_users":num_of_users,"game_num_of_players":game_num_of_players,"game_developer":game_developer})
    df=df.append(df_1)
    game_title = []
    release_year =[]
    num_of_critics=[]
    critics_grade=[]
    game_summary = []
    user_grade=[]
    platform = []
    num_of_users = []
    game_num_of_players = []
    game_developer = []
    time.sleep(5)
    print(counter)

In [ ]:
#strip and export

In [ ]:
df['num_of_critics']=df['num_of_critics'].str.strip()
df.to_csv('all_games_df')